Skipping frames

In [39]:
import os
import io
import imageio
import medmnist
import ipywidgets
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import cv2
import random 

In [40]:
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.8.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [41]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [42]:
IMG_SIZE = 128
MAX_SEQ_LENGTH = 724
frame_step = 2

# DATA
BATCH_SIZE = 6
AUTO = tf.data.AUTOTUNE
INPUT_SHAPE = (MAX_SEQ_LENGTH, IMG_SIZE, IMG_SIZE, 3)
NUM_CLASSES = 1

# OPTIMIZER
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-5

# TRAINING
EPOCHS = 25

# TUBELET EMBEDDING
PATCH_SIZE = (200, 32, 32)
#NUM_PATCHES = (INPUT_SHAPE[0] // PATCH_SIZE[0]) ** 2

# ViViT ARCHITECTURE
LAYER_NORM_EPS = 1e-6
PROJECTION_DIM = 128
NUM_HEADS = 8
NUM_LAYERS = 8

In [43]:
df1 = pd.read_csv("../data/female-mirror-data.csv")
df1["Action"] = df1["Action"].str.rstrip()
df1.head()

,Female Participant Number,Action,video-name,Unnamed: 3
0,1,Normal,1-FemaleNoGlasses-Normal.avi,NaN
1,1,Talking,1-FemaleNoGlasses-Talking.avi,NaN
2,1,Yawning,1-FemaleNoGlasses-Yawning.avi,NaN
3,2,Normal,2-FemaleNoGlasses-Normal.avi,C:\Users\bnich\Documents\_MyDocs\_School\2022-...
4,2,Talking,2-FemaleNoGlasses-Talking.avi,C:\Users\bnich\Documents\_MyDocs\_School\2022-...


In [44]:
df1["target"] = (df1["Action"]=="Yawning")

In [45]:
i = 0
dfTrain = pd.DataFrame()
dfTest = pd.DataFrame()

while i<len(df1):
    if i%5==0:
        dfTest = pd.concat([dfTest, df1.iloc[[i]]])
    else :
        dfTrain = pd.concat([dfTrain, df1.iloc[[i]]])

    i+=1

In [46]:
dfTrain.shape[0]

124

In [47]:
dfTest.shape[0]

32

In [48]:
dfTrain[dfTrain.target == True].shape[0]

42

In [49]:
dfTest[dfTest.target == True].shape[0]

8

In [50]:
def format_frames(frame, output_size):
  """
    Pad and resize an image from a video.

    Args:
      frame: Image that needs to resized and padded. 
      output_size: Pixel size of the output frame image.

    Return:
      Formatted frame with padding of specified output size.
  """
  frame = tf.image.convert_image_dtype(frame, tf.float32)
  frame = tf.image.resize_with_pad(frame, *output_size)
  return frame

In [51]:
def frames_from_video_file(video_path, MAX_SEQ_LENGTH, frame_step, output_size = (IMG_SIZE, IMG_SIZE)):
  """
    Creates frames from each video file present for each category.

    Args:
      video_path: File path to the video.
      n_frames: Number of frames to be created per video file.
      output_size: Pixel size of the output frame image.

    Return:
      An NumPy array of frames in the shape of (n_frames, height, width, channels).
  """
  # Read each video frame by frame
  result = []
  
  src = cv2.VideoCapture(str(video_path))  

  video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

  need_length = 1 + (MAX_SEQ_LENGTH - 1) * frame_step

  if need_length > video_length:
    start = 0
  else:
    max_start = video_length - need_length
    start = random.randint(0, max_start + 1)

  src.set(cv2.CAP_PROP_POS_FRAMES, start)
  # ret is a boolean indicating whether read was successful, frame is the image itself
  ret, frame = src.read()
  result.append(format_frames(frame, output_size))

  for _ in range(MAX_SEQ_LENGTH - 1):
    for _ in range(frame_step):
      ret, frame = src.read()
    if ret:
      frame = format_frames(frame, output_size)
      result.append(frame)
    else:
      result.append(np.zeros_like(result[0]))
  src.release()
  result = np.array(result)[..., [2, 1, 0]]

  return result

In [52]:
class FrameGenerator:
  def __init__(self, df, root_dir, MAX_SEQ_LENGTH, frame_step):
    """ Returns a set of frames with their associated label. 

      Args:
        path: Video file paths.
        n_frames: Number of frames. 
        training: Boolean to determine if training dataset is being created.
    """
    self.df = df
    self.n_frames = MAX_SEQ_LENGTH
    self.root_dir = root_dir
    self.frame_step = frame_step

  def __call__(self):
    video_paths = self.df["video-name"].values.tolist()
    labels = self.df["target"].values.tolist()

    for idx, path in enumerate(video_paths):
      video_frames = frames_from_video_file(os.path.join(self.root_dir, path), self.n_frames, frame_step) 
      yield video_frames, labels[idx]

In [53]:
output_signature = (tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
                    tf.TensorSpec(shape = (), dtype = tf.bool))
train_ds = tf.data.Dataset.from_generator(FrameGenerator(dfTrain, "../data/YawDD/YawDD dataset/Mirror/Female_mirror/", MAX_SEQ_LENGTH, frame_step),  output_signature = output_signature)

In [54]:
test_ds = tf.data.Dataset.from_generator(FrameGenerator(dfTest, "../data/YawDD/YawDD dataset/Mirror/Female_mirror/", MAX_SEQ_LENGTH, frame_step),  output_signature = output_signature)

In [55]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)

In [56]:
train_ds = train_ds.batch(BATCH_SIZE)
test_ds = test_ds.batch(BATCH_SIZE)

In [57]:
class TubeletEmbedding(layers.Layer):
    def __init__(self, embed_dim, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.projection = layers.Conv3D(
            filters=embed_dim,
            kernel_size=patch_size,
            strides=patch_size,
            padding="VALID",
        )
        self.flatten = layers.Reshape(target_shape=(-1, embed_dim))

    def call(self, videos):
        projected_patches = self.projection(videos)
        flattened_patches = self.flatten(projected_patches)
        return flattened_patches

In [58]:
class PositionalEncoder(layers.Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim

    def build(self, input_shape):
        _, num_tokens, _ = input_shape
        self.position_embedding = layers.Embedding(
            input_dim=num_tokens, output_dim=self.embed_dim
        )
        self.positions = tf.range(start=0, limit=num_tokens, delta=1)

    def call(self, encoded_tokens):
        # Encode the positions and add it to the encoded tokens
        encoded_positions = self.position_embedding(self.positions)
        encoded_tokens = encoded_tokens + encoded_positions
        return encoded_tokens

In [59]:
def create_vivit_classifier(
    tubelet_embedder,
    positional_encoder,
    input_shape=INPUT_SHAPE,
    transformer_layers=NUM_LAYERS,
    num_heads=NUM_HEADS,
    embed_dim=PROJECTION_DIM,
    layer_norm_eps=LAYER_NORM_EPS,
    num_classes=NUM_CLASSES,
):
    # Get the input layer
    inputs = layers.Input(shape=(input_shape))
    # Create patches.
    patches = tubelet_embedder(inputs)
    # Encode patches.
    encoded_patches = positional_encoder(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization and MHSA
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim // num_heads, dropout=0.1
        )(x1, x1)

        # Skip connection
        x2 = layers.Add()([attention_output, encoded_patches])

        # Layer Normalization and MLP
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = keras.Sequential(
            [
                layers.Dense(units=embed_dim * 4, activation=tf.nn.gelu),
                layers.Dense(units=embed_dim, activation=tf.nn.gelu),
            ]
        )(x3)

        # Skip connection
        encoded_patches = layers.Add()([x3, x2])

    # Layer normalization and Global average pooling.
    representation = layers.LayerNormalization(epsilon=layer_norm_eps)(encoded_patches)
    representation = layers.GlobalAvgPool1D()(representation)

    # Classify outputs.
    outputs = layers.Dense(units=num_classes, activation="sigmoid")(representation)

    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [60]:
def run_experiment():
    # Initialize model
    model = create_vivit_classifier(
        tubelet_embedder=TubeletEmbedding(
            embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
        ),
        positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
    )

    # Compile the model with the optimizer, loss function
    # and the metrics.
    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",
        metrics=[
            "accuracy"
        ],
    )

    # Train the model.
    _ = model.fit(train_ds, epochs=EPOCHS, validation_data = test_ds)

    _, accuracy = model.evaluate(test_ds)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    #print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return model

In [61]:
trained_model = run_experiment()

Epoch 1/25
21/21 [==============================] - 1123s 21s/step - loss: 0.6847 - accuracy: 0.6371 - val_loss: 0.5765 - val_accuracy: 0.6875
Epoch 2/25
21/21 [==============================] - 95s 5s/step - loss: 0.5732 - accuracy: 0.7016 - val_loss: 0.6201 - val_accuracy: 0.7188
Epoch 3/25
21/21 [==============================] - 130s 6s/step - loss: 0.5477 - accuracy: 0.6935 - val_loss: 0.6007 - val_accuracy: 0.7188
Epoch 4/25
21/21 [==============================] - 157s 8s/step - loss: 0.5311 - accuracy: 0.6855 - val_loss: 0.6035 - val_accuracy: 0.7188
Epoch 5/25
21/21 [==============================] - 96s 5s/step - loss: 0.5154 - accuracy: 0.7016 - val_loss: 0.6687 - val_accuracy: 0.7188
Epoch 6/25
21/21 [==============================] - 104s 5s/step - loss: 0.5082 - accuracy: 0.7016 - val_loss: 0.6922 - val_accuracy: 0.7188
Epoch 7/25
21/21 [==============================] - 123s 6s/step - loss: 0.4933 - accuracy: 0.7823 - val_loss: 0.6255 - val_accuracy: 0.7188
Epoch 8/25
21

In [62]:
NUM_SAMPLES_VIZ = 4
t = iter(test_ds)
for i in range(1):
    testsamples, labels = t.get_next()
testsamples, labels = testsamples[:NUM_SAMPLES_VIZ], labels[:NUM_SAMPLES_VIZ]

ground_truths = []
preds = []
videos = []

for i, (testsample, label) in enumerate(zip(testsamples, labels)):
    # Generate gif
    with io.BytesIO() as gif:
        imageio.mimsave(gif, (testsample.numpy()*255).astype("uint8"), "GIF")
        videos.append(gif.getvalue())

    # Get model prediction
    output = trained_model.predict(tf.expand_dims(testsample, axis=0))[0]
    pred = np.argmax(output, axis=0)

    ground_truths.append(label.numpy().astype("int"))
    preds.append(pred)


def make_box_for_grid(image_widget, fit):
    """Make a VBox to hold caption/image for demonstrating option_fit values.

    Source: https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html
    """
    # Make the caption
    if fit is not None:
        fit_str = "'{}'".format(fit)
    else:
        fit_str = str(fit)

    h = ipywidgets.HTML(value="" + str(fit_str) + "")

    # Make the green box with the image widget inside it
    boxb = ipywidgets.widgets.Box()
    boxb.children = [image_widget]

    # Compose into a vertical box
    vb = ipywidgets.widgets.VBox()
    vb.layout.align_items = "center"
    vb.children = [h, boxb]
    return vb


boxes = []
#class_vocab = ["Not Yawning", "Yawning"]

for i in range(NUM_SAMPLES_VIZ):
    ib = ipywidgets.widgets.Image(value=videos[i], width=100, height=100)
    true_class = str(ground_truths[i])
    pred_class = str(preds[i])
    caption = f"T: {true_class} | P: {pred_class}"

    boxes.append(make_box_for_grid(ib, caption))

ipywidgets.widgets.GridBox(
    boxes, layout=ipywidgets.widgets.Layout(grid_template_columns="repeat(5, 200px)")
)

GridBox(children=(VBox(children=(HTML(value="'T: 0 | P: 0'"), Box(children=(Image(value=b'GIF89a\x80\x00\x80\x…